In [ ]:
import sqlalchemy
from sqlalchemy import create_engine, Column, Integer, String, BigInteger, Sequence, ForeignKey
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
from typing import List
from typing import Optional
from sqlalchemy.orm import Mapped
from sqlalchemy.orm import mapped_column
from sqlalchemy.orm import relationship
import os
from libv3.utils import *
import pandas as pd
import json
import shutil

# Define the base for the declarative model
Base = declarative_base()

# Define the Event class which will be mapped to the events table in the database
class Event(Base):
    __tablename__ = 'events'
    
    id: Mapped[int] = mapped_column(primary_key=True)
    
    name: Mapped[int] = mapped_column(nullable=False)
    timestamp: Mapped[int] = mapped_column(BigInteger, nullable=False)
    file_number: Mapped[int] = mapped_column(ForeignKey("file_config.id")) 

    config: Mapped["File_config"] = relationship(back_populates="events")
    


class File_config(Base):
    __tablename__ = 'file_config'

    id: Mapped[int] = mapped_column(primary_key=True)
    unique_name: Mapped[str] = mapped_column(String(50), nullable=False, unique=True)
    code_base: Mapped[str] = mapped_column(String(50), nullable=False)
    version: Mapped[int] = mapped_column(Integer, nullable=False)
    behaviour: Mapped[str] = mapped_column(String(50), nullable=False)
    trial_num: Mapped[int] = mapped_column(Integer, nullable=False)

    events: Mapped[List["Event"]] = relationship(back_populates="config")


def get_or_create(code_base, version, behaviour, trial_num):
    '''
    Function to get or create a new file_config entry in
    the database. This function will return the existing
    entry if it exists, otherwise it will create a new entry
    and return it.

    Args:
        code_base (str): The code base used to generate the data
        version (int): The version of the code base
        behaviour (str): The behaviour being tested
        trial_num (int): The trial number of the data

    Returns:
        File_config: The file_config entry
        bool: False if the entry was created, True if the entry already existed
    '''
    with Session() as session:
        instance = session.query(File_config).filter_by(unique_name=f'{code_base}_{version}_{behaviour}_{trial_num}').first()
        if instance:
            return instance, True  # Return existing entry
        else:
            new_config = File_config(code_base=code_base,version=version, behaviour=behaviour, trial_num=trial_num, unique_name=f'{code_base}_{version}_{behaviour}_{trial_num}')
            session.add(new_config)
            session.commit()
            return new_config, False  # New config added
        
def check_event_exists(name, timestamp, unique_name):
    '''
    Function to check if an event already exists in the database

    Args:
        name (str): The name of the event
        timestamp (int): The timestamp of the event
        file_number (int): The file number of the event

    Returns:
        bool: True if the event exists, False if it does not
    '''
    with Session() as session:
        file_number = session.query(File_config).filter_by(unique_name=unique_name).first().id
        instance = session.query(Event).filter_by(name=name, timestamp=timestamp, file_number=file_number).first()
        if instance:
            return True
        else:
            return False



In [ ]:
############ configuration ################
############################################

CODE, BEHAVIOUR, THREAD, _ = get_config('mamba2_config')   ### config stored in libv3/exp_config.txt
VER = 4
TRIAL = 4

base_dir = '../trace_data' ### can be replaced with 'csv', 'exe_plot', 'histogram'
log_path = base_dir+f'/{CODE}/{THREAD}_thread/version_{VER}/{BEHAVIOUR}/trial{TRIAL}'

print(log_path)

#### file to display
trace_file = 0

print('file number:', trace_file)

In [ ]:
######### get paths #######################
# paths_log, paths_traces, varlist_path, paths_label = get_paths(log_path)

# ### remove.Ds_store from all lists
# paths_log = [x for x in paths_log if '.DS_Store' not in x]
# paths_traces = [x for x in paths_traces if '.DS_Store' not in x]
# varlist_path = [x for x in varlist_path if '.DS_Store' not in x]
# paths_label = [x for x in paths_label if '.DS_Store' not in x]

# paths_log.sort()
# paths_traces.sort()
# varlist_path.sort()

# print(paths_log)
# print(paths_traces)
# print(varlist_path)
# print(paths_label)

### Load the existing files

In [ ]:
### load combined trace file and varlist
all_content = read_traces(f'/Users/saurabh/Documents/Nextcloud/work/Phd tasks/diagnosis tool/trace_data/{CODE}/{THREAD}_thread/version_{VER}/{BEHAVIOUR}/trace_trial{TRIAL}')
varlist_content = read_json(f'/Users/saurabh/Documents/Nextcloud/work/Phd tasks/diagnosis tool/trace_data/{CODE}/{THREAD}_thread/version_{VER}/{BEHAVIOUR}/varlist_trial{TRIAL}.json')

In [ ]:
len(all_content)

In [ ]:
len(varlist_content)

In [ ]:
# path_to_content = os.path.dirname(log_path)+f'/trace_trial{TRIAL}'
# content_towrite = read_traces(path_to_content)
# print(content_towrite)
# Create an SQLite database (or connect to it if it already exists)
database_url = 'sqlite:///events.db'
engine = create_engine(database_url, echo=True)

# Create all tables in the database
Base.metadata.create_all(engine)

# # Create a configured "Session" class
Session = sessionmaker(bind=engine)

config_settings, config_exists = get_or_create(code_base=CODE, version=VER, behaviour=BEHAVIOUR, trial_num=TRIAL)
print(config_settings, config_exists)

if not config_exists:
    with Session.begin() as session:
        for event_name, event_timestamp in all_content:
            print(event_name, event_timestamp)

            # event_exists = check_event_exists(event_name, event_timestamp, f'{CODE}_{VER}_{BEHAVIOUR}_{TRIAL}')
            # if event_exists:
            #     print(f"Event {event_name} at timestamp {event_timestamp} already exists in the database")
            #     break

            event = Event(name=event_name, timestamp=event_timestamp, config=config_settings)
            session.add(event)
else:
    print("Config already exists in the database")


In [ ]:
#### query the database


with Session.begin() as session:
    results = session.query(File_config).all()
    for i in results:
        print(i.code_base, i.version, i.behaviour, i.trial_num, i.id)

In [ ]:
#### delete entries

# to_delete = 35

# with Session.begin() as session:
#     results = session.query(Event).all()
#     file_deleted = False
#     for i in results:
#         if i.file_number == to_delete:
#             print(i.name, i.timestamp, i.file_number, 'deleted')
#             session.delete(i)
#             file_deleted = True

#     if not file_deleted:
#         print('File config does not exist')

# with Session.begin() as session:
#     results = session.query(File_config).all()
#     event_deleted = False
#     for i in results:
#         if i.id == to_delete:
#             print(i.code_base, i.version, i.behaviour, i.trial_num, i.id, 'deleted')
#             session.delete(i)
#             event_deleted = True
            
#     if not event_deleted:
#         print('Event does not exist')